# Artificial and Computational Intelligence (ACI): Assignment 1

#### Assignment Contributor



This assignment is submitted by: **Group 306**.

The team includes:

| S.no.          | Name           | BITS ID        | Contribution   |
|----------------|----------------|----------------|----------------|
| 1              | Vidushi Bhatia | 2024ac05012    | 100%           |

<br>

<hr>

<br>

#### Assigned Problem Statement

Given the below maze configuration, the task of the robot is to navigate in the maze and find
the optimal path to reach the finish position. It can move to the north, south, west and east
direction. While navigating through the environment it has obstacles like walls. For each
transition, a path cost of +3 is added in search. Assume that the robot’s vision sensors are
sensitive to the exposure to the sunlight and whenever it tries to move towards the east direction
resulting in incurring an additional penalty of +5 cost. Use Manhattan distance as a heuristic
wherever necessary.

<img src = "maze-problem.png" width = 50%>

<br>

a. Explain the PEAS (Performance measure, Environment, Actuator, Sensor.) for
your agent. (2 Marks)

b. Implement A* algorithm using python. Use the below combination and
interpret your observations (6 Marks)

`f(n) = g(n) + w.h(n)`

Scenario 1
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 1.0
- Observation Focus: Optimal path guaranteed, slower due to full exploration.

Scenario 2
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 2.5
- Observation Focus: Faster search with fewer nodes expanded, but path may be suboptimal

Carefully read the question and submit your individual response using this
form: (5 Marks)

<br>

<hr>

<br>

## Problem Solution

#### 1.	Define the environment in the following block

<u>**PEAS Description**</u>

**P - Performance Metrics**
- Total path cost - Use the most efficient (optimal) path according to the defined costs. - minimize
- Number of collisions with walls - minimize
- Number of step to reach the finish point - minimize
- Number of steps taken in the east direction - minimize
- time taken to find the finish position - minimize


**E - Environment**
- a maze with gird layout
- Clearly defined start and finish positions start and finish positions - 
- walls/obstacles Fixed walls that act as obstacles and block movement.
- Sunlight exposure
- open path to traverse 
- Static, observable layout

**A - Actuators**
- movement control/steering ability in 4 directions
- ability to traverse - robot legs/wheels
- Mechanisms to stop or change direction to avoid walls.

**S - Sensors**
- sensors to detect walls (Proximity/vision)
- sensors to dectect sunlight (thermal/UV)
- ability to detect finish state

<br>
<hr>
<br>

<u>**Given Problem Formulation**</u>

**Initial State**
The agent starts at the given Start position.

**Actions**
- move: north, south, west and east direction
- can't move through obstacles like walls

**State Space**
 State Representation -- A state is represented as the current cell coordinates: (x, y).

**Transition Model**
Given a state (x, y) and an action, the agent transitions to a new state (x', y') if the move is legal.
	•	Each state represents the robot’s current position (row, column) in the grid.
    •	The robot can be in any free cell that is not blocked by a wall within the maze boundaries.

**Goal Test**
Find the optimal path to reach the finish position from the given initial state with minimum path cost and minimum manhnttan distance heuristic.
The goal test checks if the current state equals the goal coordinates.


**Path Cost**
- for each transition, +3
- for each move towards east, additional +5

<br>
<hr>
<br>



In [ ]:
# Code Block : Set Initial State (Must handle dynamic inputs)

def get_initial_and_goal_positions(maze):
    start = None
    goal = None

    for i in range(len(maze)):
        for j in range(len(maze[0])):
            if maze[i][j] == -99:
                start = (i, j)
            elif maze[i][j] == 99:
                goal = (i, j)
    
    if not start or not goal:
        raise ValueError("Start or Goal not defined in maze.")
    
    return start, goal

def set_initial_state(maze):
    """Find the 'Start' position in the maze; assumes 'S' denotes Start."""
    for row in range(len(maze)):
        for col in range(len(maze[0])):
            if maze[row][col] == 'S':
                return (row, col)
    raise ValueError("Start position not found.")




In [ ]:
# Code Block : Set the matrix for transition & cost (as relevant for the given problem)

maze_transition = [
    [-99, 1,   0,   0,   0,   0],
    [0,   1,   0,   1,   1,   0],
    [0,   0,   0,   1,   0,   0],
    [1,   1,   0,   1,   0,   1],
    [0,   0,   0,   0,   0,  99]
]

# Move costs
MOVE_COST = 3
SUNLIGHT_PENALTY = 5

def manhattan(current, goal):
    return abs(current[0] - goal[0]) + abs(current[1] - goal[1])


# Example dynamic maze matrix
maze = [
    ['S', '0', '1', '0', '0', '0', '0'],
    ['1', '0', '1', '0', '1', '1', '0'],
    ['1', '0', '0', '0', '1', '0', '1'],
    ['0', '0', '1', '0', '1', '0', 'F']
]

# Movement costs: direction : (delta_row, delta_col, cost)
actions = {
    'N': (-1, 0, 3),
    'S': (1, 0, 3),
    'E': (0, 1, 8),    # 3 (move) + 5 (east penalty)
    'W': (0, -1, 3)
}


def manhattan_distance(state, maze):
    """Find goal location dynamically and compute Manhattan distance."""
    rows, cols = len(maze), len(maze[0])
    for row in range(rows):
        for col in range(cols):
            if maze[row][col] == 'F':
                goal = (row, col)
                return abs(state[0] - goal[0]) + abs(state[1] - goal[1])
    raise ValueError("Goal position not found.")

In [ ]:
# Code Block : Write function to design the Transition Model/Successor function. 
# Ideally this would be called while search algorithms are implemented

def get_successors(position, maze):
    successors = []
    x, y = position
    rows, cols = len(maze), len(maze[0])

    directions = {
        "N": (-1, 0),
        "S": (1, 0),
        "W": (0, -1),
        "E": (0, 1)
    }

    for action, (dx, dy) in directions.items():
        nx, ny = x + dx, y + dy
        if 0 <= nx < rows and 0 <= ny < cols and maze[nx][ny] != 1:
            cost = MOVE_COST
            if action == "E":
                cost += SUNLIGHT_PENALTY
            successors.append(((nx, ny), cost))
    
    return successors

def get_successors(state, maze, actions):
    """Return list of (successor_state, action, cost) from current state."""
    successors = []
    rows, cols = len(maze), len(maze[0])
    r, c = state
    for action in actions:
        dr, dc, move_cost = actions[action]
        nr, nc = r + dr, c + dc
        if 0 <= nr < rows and 0 <= nc < cols and maze[nr][nc] != '1':
            successors.append(((nr, nc), action, move_cost))
    return successors


In [ ]:
# Code block : Write fucntion to handle goal test (Must handle dynamic inputs). 
# Ideally this would be called while search algorithms are implemented

def is_goal(state, goal_state):
    return state == goal_state


def is_goal_state(state, maze):
    """Check if the current state is the goal."""
    r, c = state
    return maze[r][c] == 'F'

### 2. A* algorithm

`f(n) = g(n) + w.h(n)`

Scenario 1
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 1.0
- Observation Focus: Optimal path guaranteed, slower due to full exploration.
Scenario 2
- Heuristic: Manhattan Distance
- Heuristic Weight (w): 2.5
- Observation Focus: Faster search with fewer nodes expanded, but path may be suboptimal



In [2]:
# Code Block : Function for algorithm 1 implementation

import heapq

import heapq

def a_star_search(maze, w=1.0):
    start, goal = get_initial_and_goal_positions(maze)

    def heuristic(pos):
        return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

    open_list = []
    heapq.heappush(open_list, (0, 0, start))  # (f(n), g(n), position)
    
    came_from = {}
    cost_so_far = {start: 0}
    nodes_expanded = 0

    while open_list:
        print("\n====== STEP", nodes_expanded + 1, "======")
        print("Open List Before Expansion:")
        for entry in open_list:
            print(f"  State: {entry[2]}, f(n): {entry[0]:.1f}, g(n): {entry[1]}")

        _, g, current = heapq.heappop(open_list)
        print(f"\n🔍 Expanding Node: {current} with g(n)={g}")

        nodes_expanded += 1

        if is_goal(current, goal):
            print("✅ Goal Reached!")
            path = reconstruct_path(came_from, current)
            total_cost = cost_so_far[current]
            return {
                "path": path,
                "cost": total_cost,
                "nodes_expanded": nodes_expanded
            }

        successors = get_successors(current, maze)
        print("\nNext Possible States:")
        for neighbor, cost in successors:
            print(f"  ➤ {neighbor} with step cost: {cost}")

        for neighbor, step_cost in successors:
            new_cost = cost_so_far[current] + step_cost
            if neighbor not in cost_so_far or new_cost < cost_so_far[neighbor]:
                cost_so_far[neighbor] = new_cost
                f_score = new_cost + w * heuristic(neighbor)
                heapq.heappush(open_list, (f_score, new_cost, neighbor))
                came_from[neighbor] = current
                print(f"  ⬆️ Added to Open List: {neighbor}, f(n)={f_score:.1f}, g(n)={new_cost}")

    print("❌ No path found.")
    return None



def reconstruct_path(came_from, current):
    path = [current]
    while current in came_from:
        current = came_from[current]
        path.append(current)
    path.reverse()
    return path


In [ ]:
import heapq

def a_star_search(maze, weight=1.0, start_sym='S', finish_sym='F', wall_sym='1'):
    """
    Performs A* search with weighted heuristic on the given maze.
    Args:
        maze: List of rows; elements as 'S' (start), 'F' (finish), '1'(wall), '0'(free cell)
        weight: Float, weight for heuristic h(n) in f(n) = g(n) + w*h(n)
        start_sym: Char, symbol for start
        finish_sym: Char, symbol for finish
        wall_sym: Char, symbol for wall
    Returns:
        path: List of (row, col) tuples from start to goal, or None if no path found
        total_cost: The cost corresponding to the path
    """    

    # Helper functions

    def get_position(sym):
        for r in range(len(maze)):
            for c in range(len(maze[0])):
                if maze[r][c] == sym:
                    return (r, c)
        raise ValueError(f"Symbol {sym} not found in maze.")

    def get_neighbors(state):
        r, c = state
        directions = {
            'N': (-1, 0, 3),   # Up
            'S': (1, 0, 3),    # Down
            'E': (0, 1, 8),    # Right (East: 3+5 penalty)
            'W': (0, -1, 3)    # Left
        }
        for dir in directions:
            dr, dc, cost = directions[dir]
            nr, nc = r + dr, c + dc
            if 0 <= nr < len(maze) and 0 <= nc < len(maze[0]):
                if maze[nr][nc] != wall_sym:
                    yield (nr, nc), cost

    def manhattan(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    # Initialize
    start = get_position(start_sym)
    goal = get_position(finish_sym)
    heap = []
    heapq.heappush(heap, (weight*manhattan(start, goal), 0, start, [start]))
    visited = set()

    while heap:
        f, g, current, path = heapq.heappop(heap)
        if current == goal:
            return path, g   # Success
        
        if current in visited:
            continue
        visited.add(current)
        
        for neighbor, step_cost in get_neighbors(current):
            if neighbor not in visited:
                new_g = g + step_cost
                h = manhattan(neighbor, goal)
                new_f = new_g + weight * h
                heapq.heappush(heap, (new_f, new_g, neighbor, path + [neighbor]))
    return None, None

# --- Example usage ---
if __name__ == "__main__":
    # Dynamic input example
    maze = [
        ['S', '0', '1', '0', '0', '0'],
        ['1', '0', '1', '0', '1', '0'],
        ['1', '0', '0', '0', '1', '0'],
        ['0', '0', '1', '0', '1', 'F']
    ]
    weight = 1.0  # Standard A* (weight=1); >1 makes it greedier, <1 makes it more conservative

    path, total_cost = a_star_search(maze, weight)
    if path is not None:
        print(f"Path found: {path}")
        print(f"Total cost: {total_cost}")
    else:
        print("No path found.")


### 3. DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from: This is applicable for all the relevent problems as mentioned in the question.

In [ ]:
#Code Block : Function & call to get inputs (start/end state)

start, goal = get_initial_and_goal_positions(maze)
print("Start:", start)
print("Goal:", goal)

successors = get_successors(start, maze)
print("Successors of start:", successors)

result = a_star_search(maze, w=1.0)


start = set_initial_state(maze)

### 4.	Calling the search algorithms

In [ ]:
#Invoke algorithm 1 (Should Print the solution, path, cost, Interpretation etc., (As mentioned in the problem))

#scenario 1
result_1 = a_star_search_no_heap(maze, w=1.0)

print("\n--- Scenario 1: A* with w=1.0 ---")
print("\n=== FINAL RESULT ===")
if result_1:
    print("Path:", result_1['path'])
    print("Total Cost:", result_1['cost'])
    print("Nodes Expanded:", result_1['nodes_expanded'])
else:
    print("No path found.")


In [ ]:
result_2 = a_star_search_no_heap(maze, w=2.5)

print("\n--- Scenario 2: A* with w=2.5 ---")
if result_2:
    print("Path:", result_2['path'])
    print("Total Cost:", result_2['cost'])
    print("Nodes Expanded:", result_2['nodes_expanded'])
    print("Interpretation: Faster but may be suboptimal due to aggressive heuristic.")
    print("Time Complexity: O(b^d) — reduced in practice due to fewer nodes explored")
    print("Space Complexity: O(b^d) — reduced open list size compared to w=1.0")
else:
    print("No path found.")

### 5.	Comparitive Analysis

In [ ]:
#Code Block : Print the Time & Space complexity of algorithm